In [6]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import d2l
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#下面进入backbone网络模块实现
#基础模块-Conv模块
class Conv(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=None,groups=1,activation=None):
        super().__init__()
        if padding is None:
            padding=kernel_size//2
        
        self.conv=nn.Conv2d(in_channels,out_channels,
                            kernel_size,stride,padding,
                            groups=groups,bias=False)
        self.bn=nn.BatchNorm2d(out_channels)
        self.act=nn.SiLU() if activation else nn.Identity()
    
    def foreard(self,X):
        return self.act(self.bn(self.conv(X)))

#瓶颈模块
class BottleNeck(nn.Module):
    def __init__(self, in_channels,out_channels,shortcut=True):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,1,1)
        self.conv2=nn.Conv2d(out_channels,out_channels,3,1)
        self.shortcut=shortcut and in_channels==out_channels
    
    def forward(self,X):
        if self.shortcut:
            return X+self.conv2(self.conv1(X))
        else:
            return self.conv2(self.conv1(X))